In [2]:
import pandas as pd
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS, RandomEffects
from linearmodels.panel import compare

# Carregar a base
df = pd.read_stata("Base1.dta")

# Definir variáveis de interesse
vars_interesse = ["EARNINGS", "UNION", "S", "HOURS", "EXP", "TENURE", "MALE", "ETHBLACK", "ETHHISP", "URBAN", "ID", "TIME"]
df = df.dropna(subset=vars_interesse)

# Definir ID e Tempo como MultiIndex
df = df.set_index(["ID", "TIME"])

# Construir a fórmula
base_vars = ["UNION", "S", "HOURS", "EXP", "TENURE", "MALE", "ETHBLACK", "ETHHISP", "URBAN"]
formula_re = "EARNINGS ~ " + " + ".join(base_vars)

# Estimar o modelo de Efeitos Aleatórios
re_model = RandomEffects.from_formula(formula_re, data=df).fit()
print("Resultados do Modelo de Efeitos Aleatórios:")
print(re_model.summary)

# Estimar o modelo de Efeitos Aleatórios com matriz de variância robusta
re_model_robust = RandomEffects.from_formula(formula_re, data=df).fit(cov_type='robust')
print("Resultados do Modelo de Efeitos Aleatórios com Matriz de Variância Robusta:")
print(re_model_robust.summary)

# Estimar FGLS (Feasible Generalized Least Squares)
fgls_model = smf.gls(formula_re, data=df).fit()
print("Resultados do FGLS:")
print(fgls_model.summary())

# Teste para a presença de efeitos não observáveis
test_result = compare({'RE': re_model, 'RE Robust': re_model_robust})
print("Teste para Efeitos Não Observáveis:")
print(test_result)

# Interpretação da comparação
print("\nInterpretação do Teste para Efeitos Não Observáveis:")
if re_model.rsquared < re_model_robust.rsquared:
    print("O modelo de RE com matriz de variância robusta apresenta um melhor ajuste, sugerindo que heterocedasticidade pode estar presente.")
else:
    print("Não há evidência clara de que a matriz de variância robusta melhore substancialmente o ajuste do modelo de RE.")

if re_model.pvalues.mean() < 0.05:
    print("A presença de efeitos não observáveis é estatisticamente significativa, indicando que o modelo pode precisar de uma abordagem mais robusta.")
else:
    print("Não há evidências estatísticas fortes para a presença de efeitos não observáveis.")


Resultados do Modelo de Efeitos Aleatórios:
                        RandomEffects Estimation Summary                        
Dep. Variable:               EARNINGS   R-squared:                        0.4974
Estimator:              RandomEffects   R-squared (Between):              0.8215
No. Observations:               24508   R-squared (Within):               0.0898
Date:                Tue, Apr 01 2025   R-squared (Overall):              0.7675
Time:                        07:55:46   Log-likelihood                -7.745e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2694.2
Entities:                        4486   P-value                           0.0000
Avg Obs:                       5.4632   Distribution:                 F(9,24499)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):    